# CSC421 Assignment 2 - Part II  First-Order Logic (5 points) #
### Author: George Tzanetakis 

This notebook is based on the supporting material for topics covered in **Chapter 7 - Logical Agents** from the book *Artificial Intelligence: A Modern Approach.* You can consult and modify the code provided in logic.py and logic.ipynb for completing the assignment questions. This part does rely on the provided code. 

```
Birds can fly, unless they are penguins and ostriches, or if they happen 
to be dead, or have broken wings, or are confined to cages, or have their 
feet stuck in cement, or have undergone experiences so dreadful as to render 
them psychologically incapable of flight 

Marvin Minsky 
```

# Introduction - First-Order Logic and knowledge engineering 

In this assignment we explore First-Order Logic (FOL) using the implementation of knowledge base and first-order inference provided by the textbook authors. We also look into matching a limited form of unification. 

**NOTE THAT THE GRADING IN THIS ASSIGNMENT IS DIFFERENT FOR GRADUATE STUDENTS AND THEY HAVE TO DO EXTRA WORK FOR FULL MARKS**


# Question 2A (Minimum) (CSC421 -  1 point, CSC581C - 0 points) 

Consider the following propositional logic knowledge base. 

* It is not sunny this afternoon and it is colder than yesterday.
* We will go swimming only if it is sunny.
* If we do not go swimming then we will take a canoe trip.
* If we take a canoe trip, then we will be home by sunset.


Denote:


* p = It is sunny this afternoon
* q = it is colder than yesterday
* r = We will go swimming
* s= we will take a canoe trip
* t= We will be home by sunset

Express this knowledge base using propositional logic using the expression syntax used in logic.ipynb. You can incoprorate any code you need from logic.ipynb and logic.py. In order to access the associated code the easiest way is to place your notebook in the same folder as the aima_python source code. Using both model checking and theorem proving inference (you can use the implementations provided) show that this knowledge base entails the sentence if it is not sunny this afternoon then we will be home by sunset. 

In [ ]:
from utils import *
from logic import *
from notebook import psource

In [ ]:
# YOUR CODE GOES HERE 
knowledge_base = PropKB()
(P, Q, R, S, T) = symbols("P, Q, R, S, T")

sentence1 = expr('R ==> P')
sentence2 = expr('~R ==> S')
sentence3 = expr('S ==> T')
sentence4 = expr('~P & Q')


knowledge_base.tell(sentence1)
knowledge_base.tell(sentence2)
knowledge_base.tell(sentence3)
knowledge_base.tell(sentence4)

#knowledge_base.clauses
knowledge_base.ask_if_true(expr('~P ==> T'))


# Question 2B (Minimum) (CSC421 - 1 point, CSC581C - 0 point) 

Encode the kindship domain described in section 8.3.2 of the textbook using FOL and FolKB implementation in logic.ipynb and encode as facts the relationships between the members of the Simpsons family from the popular TV show:  

https://en.wikipedia.org/wiki/Simpson_family


Show how the following queries can be answered using the KB: 

* Who are the children of Homer ? 
* Who are the parents of Bart ? 
* Are Lisa and Homer siblings ? 
* Are Lisa and Bart siblings ? 


In [ ]:
# YOUR CODE GOES HERE 
'''
Parent(x,y): x is a parent to y
Child(x,y): x is a child of y
Sibling(x,y): x and y are siblings
'''

# Define Clauses
clauses = []

# 1.
clauses.append(expr("Female(p) & Parent(p, c) ==> Child(c,p)"))
clauses.append(expr("Mother(p) ==> Female(p)"))

# 2. 
clauses.append(expr("Male(m) & Spouse(m, f) ==> Husband(m,f)"))
clauses.append(expr("Spouse(m, f) ==> Male(m)"))
clauses.append(expr("Spouse(m, f) ==> Female(f)"))

# 3.

# 4.
clauses.append(expr("Parent(p, c) ==> Child(c,p)"))
clauses.append(expr("Child(c, p) ==> Parent(p,c)"))

# 5.
clauses.append(expr("(Parent(p,x) & Parent(p,y)) ==> Sibling(x, y)"))
clauses.append(expr("(Sibling(x, y)) ==> (Sibling(y,x))"))


# Init First Order Logic KB
fam_kb = FolKB(clauses)

# Tell KB who the parents are
fam_kb.tell(expr("Parent(Homer, Bart)"))
fam_kb.tell(expr("Parent(Marge, Bart)"))
fam_kb.tell(expr("Parent(Homer, Lisa)"))

# Ask KB Question 
homers_children = fol_fc_ask(fam_kb, expr('Child(x, Homer)'))
print("Homers Children: ",list(homers_children))

barts_parents = fol_fc_ask(fam_kb, expr('Parent(x, Bart)'))
print("Barts Parents: ",list(barts_parents))

siblings = fol_fc_ask(fam_kb, expr('Sibling(Lisa, Homer)'))
print("Lisa - Homer Sibs? ",bool(list(siblings))) # Check if list empty

siblings = fol_fc_ask(fam_kb, expr('Sibling(Lisa, Bart)'))
print("Lisa - Bart Sibs? ",bool(list(siblings))) # Check if list empty


# Question 2C (Expected) 1 point 


Encode the electronic circuit domain described in section 8.4.2 of your textbook using the FolKB implementation in logics.ipynb. Encode the general knowledge of the domain as well as the specific problem instance shown in Figure 8.6. Post the same queries described by the book to the inference procedure. 

In [ ]:
# YOUR CODE GOES HERE 
'''
Terms:
    Terminal(t1)
    Connected(t1, t2)
    SignalOn(t)
    SignalOff(t)
    Gate(g)
    4 Gates
    
'''
clauses = []
# 1
#clauses.append(expr("Out(x,g) ==> Terminal(Out(x,g))"))
#clauses.append(expr("In(x,g) ==> Terminal(x,g)"))

clauses.append(expr("Terminal(x, g) & Terminal(y, g) & Connected(x, y) ==> SameSignal(x, y)"))
clauses.append(expr("SameSignal(x,y) ==> SameSignal(y, x)"))
clauses.append(expr("SameSignal(x,y) & Signal(x, 1) ==> Signal(y, 1)"))
#clauses.append(expr("SameSignal(x,y) & Signal(x, 1) ==> Signal(y, 1)"))


# 2 - Signal at each terminal is either on or off...
clauses.append(expr("Signal(t,0) ==> Terminal(t)"))
clauses.append(expr("Signal(t,1) ==> Terminal(t)"))

# 3
clauses.append(expr("Connected(t1, t2) ==> Connected(t2,t1)"))

# 4 - There are 4 types of gates
clauses.append(expr("OR(g) ==> Gate(g)"))
clauses.append(expr("XOR(g) ==> Gate(g)"))
clauses.append(expr("AND(g) ==> Gate(g)"))
clauses.append(expr("NOT(g) ==> Gate(g)"))

# 5 AND Gate

clauses.append(expr("(Gate(g) & AND(g) & Signal(In(1,g), 1) & Signal(In(2,g), 1)) ==> Signal(Out(g), 1)"))
clauses.append(expr("(Gate(g) & AND(g) & Signal(In(1,g), 1) & Signal(In(2,g), 0)) ==> Signal(Out(g), 0)"))
clauses.append(expr("(Gate(g) & AND(g) & Signal(In(1,g), 0) & Signal(In(2,g), 1)) ==> Signal(Out(g), 0)"))
clauses.append(expr("(Gate(g) & AND(g) & Signal(In(1,g), 0) & Signal(In(2,g), 0)) ==> Signal(Out(g), 0)"))

# 6 OR Gate
clauses.append(expr("(Gate(g) & OR(g) & Signal(In(1,g), 1)) ==> Signal(Out(g), 1)"))
clauses.append(expr("(Gate(g) & OR(g) & Signal(In(2,g), 1)) ==> Signal(Out(g), 1)"))
clauses.append(expr("(Gate(g) & OR(g) & Signal(In(1,g), 0)) & Signal(In(2,g), 0) ==> Signal(Out(g), 0)"))

# 7 XOR
clauses.append(expr("(Gate(g) & XOR(g) & Signal(In(1,g), 1) & Signal(In(2,g), 1)) ==> Signal(Out(g), 0)"))
clauses.append(expr("(Gate(g) & XOR(g) & Signal(In(1,g), 1) & Signal(In(2,g), 0)) ==> Signal(Out(g), 1)"))
clauses.append(expr("(Gate(g) & XOR(g) & Signal(In(1,g), 0) & Signal(In(2,g), 1)) ==> Signal(Out(g), 1)"))
clauses.append(expr("(Gate(g) & XOR(g) & Signal(In(1,g), 0) & Signal(In(2,g), 0)) ==> Signal(Out(g), 0)"))

# 8 NOT Gate
clauses.append(expr("(Gate(g) & NOT(g) & Signal(In(1,g), 0)) ==> Signal(Out(g), 1)"))
clauses.append(expr("(Gate(g) & NOT(g) & Signal(In(1,g), 1)) ==> Signal(Out(g), 0)"))

# 9
clauses.append(expr("NOT(g) ==> Arity(g, 1, 1)"))
clauses.append(expr("AND(g) ==> Arity(g, 2, 1)"))
clauses.append(expr("XOR(g) ==> Arity(g, 2, 1)"))
clauses.append(expr("OR(g) ==> Arity(g, 2, 1)"))

# 10


clauses.append(expr("Out(g) ==> Arity(g, 2, 1)"))


circuit_kb = FolKB(clauses)

circuit_kb.tell(expr("XOR(X1)"))
circuit_kb.tell(expr("XOR(X2)"))
circuit_kb.tell(expr("AND(A1)"))
circuit_kb.tell(expr("AND(A2)"))
circuit_kb.tell(expr("OR(O1)"))
circuit_kb.tell(expr("OR(O1)"))

circuit_kb.tell(expr("Connected(Out(A, X1), In(A, X2))"))
circuit_kb.tell(expr("Connected(Out(A, X1), In(B, A2))"))
circuit_kb.tell(expr("Connected(Out(A, A2), In(A, O1))"))
circuit_kb.tell(expr("Connected(Out(A, A1), In(B, O1))"))




#circuit_kb.clauses


list(fol_fc_ask(circuit_kb, expr('XOR(x)')))


# QUESTION 1D (EXPECTED) 1 point

In this question we explore Prolog which is a programming language based on logic. We won't go into details but just wanted to give you a flavor of the syntax and how it connects to what we have learned. For this question you 
will NOT be using the notebook so your answer should just be the source code. We will use http://tau-prolog.org/ which is a Prolog implementation that can run in a browser. When you access the webpage there is a text window labeled try it for entering your knowledge base and under it there is a text entry field for entering your query. 

For example type in the Try it window and press enter: 

```Prolog
likes(sam, salad).
likes(dean, pie).
likes(sam, apples).
likes(dean, whiskey).
```

Then enter the query: 
```Prolog 
likes(sam,X).
```
When you press Enter once you will get X=apples. and X=salad. Note the periods at the end of each statement. 

Encode the kinship domain from question 2B in Prolog and answer the queries from 2B. Notice that in Prolog the constants start with lower case letters and the variables start with upper case letters.

Provide your code for the KB and queries using markup. See the syntax for Prolog of this cell by double clicking for editing. 



# YOUR CODE GOES HERE

```Prolog
:- use_module(library(lists)).

child(X,Y) :-
    parent(Y,X).

parent(A,B) :-
    child(B,A)

sibling(A,B) :-
	parent(X,A),
    parent(X,B).

sibling(A,B) :- 
    sibling(B,A).

spouse(A,B) :-
    spouse(B,A).
    
husband(H,W) :-
    male(H),
    spouse(H,W).

parent(marge, bart).
parent(marge, lisa).
parent(homer, bart).
parent(homer, lisa).
```
#### Goals and responses
```Prolog
child(X,homer). -> X = bart; X = lisa .
parent(X,bart). -> X = marge ; X = homer.
sibling(lisa,homer). -> false.
siblings(lisa, bart). -> true.
```

# QUESTION 1E (ADVANCED) 1 point 

Implement exercise 8.26 using the code in logic.ipynb as well as the KB you wrote for the circuit domain. 




In [ ]:
# YOUR CODE GOES HERE 


# QUESTION 1F (ADVANCED) (CSC421 - 0 points, CSC581C - 2 points)


This question explores the automatic constructions of a first-order logic knowledge base from a web resource and is more open ended than the other ones. The website https://www.songfacts.com/ contains a large variety of facts about music. Check the https://www.songfacts.com/categories link for some categories. Using selenium Python bindings https://selenium-python.readthedocs.io/ access the webpage and scrape at least three categories. Your code should scrape the information from the pages and convert it into relationships and facts in first-order logic using the syntax of expressions in logic.ipynb. Once you build your knowledge-base then write 4 non-trivial queries that show-case the expressiveness of FOL. These queries should not be possible to be answered easily using the web interface i.e they should have some logical connectives, more than one possible answer etc. 
The translation of the song facts from the web page to FOL should NOT be done by hand but using the web scraping tool you develop. You can use multiple cells in your answer. 




In [ ]:
# YOUR CODE GOES HERE 